## Setup libraries

In [67]:
from math import sqrt
import numpy as np
import pandas as pd
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [68]:
table = read_csv("NNdata.csv")
Moggio = table.loc[table.loc[:, "NameStation"] == "Moggio", :]
Moggio = Moggio.loc[:, ["Date", "Ammonia", "PM10", "PM25",
                        "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]
# Cancella su tutta la riga anche solo con un mancante
Moggio = Moggio.dropna(how="any")
Moggio = Moggio.reset_index()
Moggio = Moggio.iloc[:730, :]  # we train not using 2020, for now

Moggio.head()


,index,Date,Ammonia,PM10,PM25,Wind_speed,Wind_direction,Temperature,Rainfall
0,1,2018-01-01T00:00:00Z,1.4,5.0,4.0,0.300970,65.0,2.095833,2.4
1,4,2018-01-02T00:00:00Z,1.0,3.0,2.0,0.613183,9.0,1.670139,0.0
2,7,2018-01-03T00:00:00Z,1.0,5.0,5.0,0.539172,28.0,1.972222,0.6
3,10,2018-01-04T00:00:00Z,1.4,6.0,3.0,0.211450,21.0,3.081250,0.0
4,13,2018-01-05T00:00:00Z,1.7,7.0,8.0,0.116953,87.0,3.653472,0.0


In [69]:
#Primo quadrante
Q1 = Moggio.loc[((Moggio.Wind_direction > 0) &
                 (Moggio.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction1"] = arr1

#Secondo quadrante
Q2 = Moggio.loc[((Moggio.Wind_direction > 90) &
                 (Moggio.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Moggio.loc[((Moggio.Wind_direction > 180) &
                 (Moggio.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Moggio.loc[((Moggio.Wind_direction > 270) &
                 (Moggio.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Moggio = pd.concat([Q1,Q2,Q3,Q4])


/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_3243/1130944625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction1"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_3243/1130944625.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_3243/1130944625.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe